In [17]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/aljowharahalshabaan/miniprojectweak3/day15_real_dataset_large.csv


In [27]:
import pandas as pd
import numpy as np
path="/kaggle/input/datasets/aljowharahalshabaan/miniprojectweak3/day15_real_dataset_large.csv"
df_or=pd.read_csv(path)
print(df_or.head(),"\n")
print(df_or.info(),"\n")
print(df_or.describe(),"\n")
print(df_or.nunique().sort_values(ascending=False).head(10),"\n")


cleaning_plan = {
"age": {"type": "float", "missing": "median_imp", "outliers": "cap_99"},
"income": {"type": "float", "missing": "median_imp", "outliers": "log1p_cap_99"},
"city": {"type": "category", "clean": "canonical_city"},
"signup_time": {"type": "datetime", "tz": "UTC"}}

def clean_data_project(df):
    df1 = df.copy()
# Types
    df1["age"] = pd.to_numeric(df1["age"], errors="coerce")
    df1["income"] = pd.to_numeric(df1["income"], errors="coerce")
    df1["signup_time"] = pd.to_datetime(df1["signup_time"], errors="coerce")
    # Missing
    df1=df1.dropna(subset=["income"])
    df1["age_missing"] = df1["age"].isna().astype(int)
    df1["age"] = df1["age"].fillna(df1["age"].median())
    df1["income_missing"] = df1["income"].isna().astype(int)
    df1["income"] = df1["income"].fillna(df1["income"].median())
    df1["city_missing"]=df1["city"].isna().astype(int)
    df1["signup_missing"]=df1["signup_time"].isna().astype(int)
    #duplicates
    dups=df1.duplicated()
    df1=df1.drop_duplicates(subset=["income","city","signup_time"])
    # Outliers
    df1["income"] = df1["income"].clip(upper=df1["income"].quantile(0.99))
    df1["age"]=np.log1p(df1["age"])
    # Strings and dates
    df1["city"] = df1["city"].str.strip().str.lower()
    canonical_map = {"new york": "new york", "nyc": "new york", "ny": "new york","san francisco": "san francisco", "sanfrancisco": "san francisco"}
    df1["city_token"] = df1["city"].str.replace(" ", "", regex=False)
    df1["city_canonical"] = df1["city_token"].map(canonical_map).fillna(df1["city"])
    custom_missing = ["N/A", "NA", "not reported", "unknown", "?"]
    df1["city"]=df1["city"].replace(custom_missing,np.nan)
    df1["signup_time"] = df1["signup_time"].dt.tz_localize("UTC")
    return df1
df_clean=clean_data_project(df_or)
print("===================================\n")
print("=============after cleaning:==================")
print(df_clean.info())
print(df_clean[["age", "income"]].describe())
print(df_clean["city"].value_counts().head())
print(df_clean["signup_time"].dt.tz)
print(df_clean.isna().sum())
df_clean.to_csv("clean_data.csv")

       age    income     city signup_time
0       30   55000.0       NY  2024-01-01
1      NaN   70000.0       SF  2024-01-05
2       45       NaN       LA  not a date
3  unknown  120000.0       NY  2024/02/01
4       28   65000.0  Chicago  2024-01-15 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97 entries, 0 to 96
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          78 non-null     object 
 1   income       77 non-null     float64
 2   city         96 non-null     object 
 3   signup_time  97 non-null     object 
dtypes: float64(1), object(3)
memory usage: 3.2+ KB
None 

              income
count      77.000000
mean    77077.922078
std     18780.425303
min     45000.000000
25%     62000.000000
50%     75000.000000
75%     91000.000000
max    120000.000000 

signup_time    95
income         62
city           59
age            30
dtype: int64 


=============after cleaning:==================
<class 

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
